<a href="https://colab.research.google.com/github/adiazoq/Proyecto-Machine-Learning/blob/main/C%C3%B3digos/DESCRIPTIVO_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

print(" Librerías importadas correctamente")

 Librerías importadas correctamente


In [ ]:
# 1. Cargar los datos
# Leer base de datos de excel
from google.colab import files
uploaded = files.upload()
!ls

Saving Temperatura_Valledupar.xlsx to Temperatura_Valledupar.xlsx
df_diario_promediado.xlsx  sample_data	Temperatura_Valledupar.xlsx


In [ ]:
# Cargar el archivo Excel
# 'ruta_del_archivo.xlsx' debe ser reemplazado por la ruta real de tu archivo
df = pd.read_excel('Temperatura_Valledupar.xlsx')

# Mostrar las primeras filas para verificar la carga correcta
print("Primeras 5 filas del dataset:")
print(df.head())

# Ver información general del dataset (tipos de datos, valores nulos, etc.)
print("\nInformación del dataset:")
print(df.info())

# Ver estadísticas descriptivas de la variable ValorObservado
print("\nEstadísticas descriptivas de ValorObservado:")
print(df['ValorObservado'].describe())

Primeras 5 filas del dataset:
   CodigoEstacion  CodigoSensor         FechaObservacion          Dia   Año  \
0        28035060            68  2005 ago 01 10:00:00 AM  2005 ago 01  2005   
1        28035060            68  2005 ago 01 06:00:00 AM  2005 ago 01  2005   
2        28035060            68  2005 ago 01 07:00:00 AM  2005 ago 01  2005   
3        28035060            68  2005 ago 01 09:00:00 PM  2005 ago 01  2005   
4        28035060            68  2005 ago 01 09:00:00 AM  2005 ago 01  2005   

      Mes  ValorObservado   NombreEstacion Departamento   Municipio  \
0  agosto             0.0  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
1  agosto             0.0  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
2  agosto             0.0  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
3  agosto             0.0  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
4  agosto             0.0  FEDEARROZ - AUT        CESAR  VALLEDUPAR   

  ZonaHidrografica    Latitud   Longitud DescripcionSensor UnidadMed

In [ ]:
# =============================================================================
# PASO 1: IDENTIFICAR LA COLUMNA DE AÑO Y VERIFICAR LOS AÑOS EXISTENTES
# =============================================================================

print("=" * 70)
print("ANÁLISIS DE LA VARIABLE DE AÑO")
print("=" * 70)

# Verificar si existe una columna llamada 'Año' en el dataset
if 'Año' in df.columns:
    print(" Columna 'Año' encontrada en el dataset")
    columna_año = 'Año'
else:
    # Si no existe, buscar columnas que puedan contener el año
    columnas_posibles = [col for col in df.columns if 'año' in col.lower() or 'year' in col.lower()]
    if columnas_posibles:
        columna_año = columnas_posibles[0]
        print(f" Columna de año identificada: '{columna_año}'")
    else:
        # Si no hay columna de año, intentar extraer de la columna 'Dia'
        print(" No se encontró columna 'Año'. Intentando extraer de columna 'Dia'...")
        # Convertir la columna Dia a datetime y extraer el año
        df['Año'] = pd.to_datetime(df['Dia'], errors='coerce').dt.year
        columna_año = 'Año'
        print(" Año extraído de la columna 'Dia'")

# Mostrar los años únicos presentes en el dataset
años_unicos = sorted(df[columna_año].unique())
print(f"\nAños presentes en el dataset original: {años_unicos}")
print(f"Rango de años: {min(años_unicos)} - {max(años_unicos)}")
print(f"Total de años diferentes: {len(años_unicos)}")

ANÁLISIS DE LA VARIABLE DE AÑO
 Columna 'Año' encontrada en el dataset

Años presentes en el dataset original: [np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
Rango de años: 2005 - 2025
Total de años diferentes: 21


In [ ]:
# =============================================================================
# PASO 2: FILTRAR EL DATASET PARA CONSERVAR SOLO 2006-2024
# =============================================================================

def filtrar_por_año(dataframe, columna_año, año_min=2006, año_max=2024):
    """
    Filtra el DataFrame para conservar solo los registros dentro del rango de años especificado

    Parámetros:
    - dataframe: DataFrame original a filtrar
    - columna_año: Nombre de la columna que contiene el año
    - año_min: Año mínimo a incluir (inclusive)
    - año_max: Año máximo a incluir (inclusive)

    Retorna:
    - DataFrame filtrado
    """

    print("=" * 70)
    print("FILTRADO POR RANGO DE AÑOS")
    print("=" * 70)

    # Crear una copia del DataFrame original
    df_filtrado = dataframe.copy()

    # Mostrar estadísticas antes del filtrado
    total_original = len(df_filtrado)
    print(f"Total de registros antes del filtrado: {total_original:,}")

    # Aplicar el filtro por rango de años
    mascara_años = (df_filtrado[columna_año] >= año_min) & (df_filtrado[columna_año] <= año_max)
    df_filtrado = df_filtrado[mascara_años]

    # Mostrar estadísticas después del filtrado
    total_filtrado = len(df_filtrado)
    registros_eliminados = total_original - total_filtrado
    porcentaje_eliminado = (registros_eliminados / total_original) * 100

    print(f"Año mínimo incluido: {año_min}")
    print(f"Año máximo incluido: {año_max}")
    print(f"Total de registros después del filtrado: {total_filtrado:,}")
    print(f"Registros eliminados: {registros_eliminados:,} ({porcentaje_eliminado:.1f}%)")

    # Verificar los años que quedaron después del filtrado
    años_restantes = sorted(df_filtrado[columna_año].unique())
    print(f"Años en el dataset filtrado: {años_restantes}")

    return df_filtrado

# Aplicar el filtrado
df_filtrado = filtrar_por_año(df, columna_año, 2006, 2024)

FILTRADO POR RANGO DE AÑOS
Total de registros antes del filtrado: 828,369
Año mínimo incluido: 2006
Año máximo incluido: 2024
Total de registros después del filtrado: 636,445
Registros eliminados: 191,924 (23.2%)
Años en el dataset filtrado: [np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]


In [ ]:
# =============================================================================
# PASO 3: VERIFICACIÓN DEL DATASET FILTRADO
# =============================================================================

print("=" * 70)
print("VERIFICACIÓN DEL DATASET FILTRADO (2006-2024)")
print("=" * 70)

# Mostrar las primeras filas del dataset filtrado
print("Primeras 5 filas del dataset filtrado:")
print(df_filtrado.head())

# Ver información general del dataset filtrado
print("\nInformación del dataset filtrado:")
print(df_filtrado.info())

# Ver estadísticas descriptivas de la variable ValorObservado en el dataset filtrado
print("\nEstadísticas descriptivas de ValorObservado (filtrado 2006-2024):")
print(df_filtrado['ValorObservado'].describe())

VERIFICACIÓN DEL DATASET FILTRADO (2006-2024)
Primeras 5 filas del dataset filtrado:
      CodigoEstacion  CodigoSensor         FechaObservacion          Dia  \
3375        28035060            68  2006 abr 01 03:00:00 AM  2006 abr 01   
3376        28035070            68  2006 abr 01 02:00:00 PM  2006 abr 01   
3377        28035070            68  2006 abr 01 04:00:00 AM  2006 abr 01   
3378        28035060            68  2006 abr 01 07:00:00 AM  2006 abr 01   
3379        28035070            68  2006 abr 01 01:00:00 PM  2006 abr 01   

       Año    Mes  ValorObservado   NombreEstacion Departamento   Municipio  \
3375  2006  abril            25.1  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
3376  2006  abril            24.6  GUATAPURI - AUT        CESAR  VALLEDUPAR   
3377  2006  abril            15.4  GUATAPURI - AUT        CESAR  VALLEDUPAR   
3378  2006  abril            25.8  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
3379  2006  abril            26.5  GUATAPURI - AUT        CESAR

In [ ]:
# =============================================================================
# CORRECCIÓN: RESETEAR ÍNDICES ANTES DE PROCESAR
# =============================================================================

print("=" * 70)
print("PREPARANDO DATASET FILTRADO PARA PROCESAMIENTO")
print("=" * 70)

# Resetear los índices del DataFrame filtrado para que sean consecutivos
df_filtrado_reset = df_filtrado.reset_index(drop=True)

print(f"Dataset original filtrado: {len(df_filtrado)} registros")
print(f"Dataset con índices reseteados: {len(df_filtrado_reset)} registros")
print("✓ Índices reseteados correctamente")

# =============================================================================
# FUNCIÓN CORREGIDA PARA REEMPLAZAR VALORES ATÍPICOS
# =============================================================================

def reemplazar_valores_atipicos_filtrado(dataframe, columna, valor_min=16, valor_max=42.5):
    """
    Esta función identifica valores atípicos en una columna y los reemplaza
    con el promedio de los 3 valores anteriores y 3 siguientes

    ADAPTADA ESPECÍFICAMENTE PARA df_filtrado (2006-2024) CON ÍNDICES RESETEADOS

    Parámetros:
    - dataframe: el DataFrame de pandas (df_filtrado_reset)
    - columna: nombre de la columna a procesar
    - valor_min: valor mínimo aceptable (default 16)
    - valor_max: valor máximo aceptable (default 42.5)
    """

    # Crear una copia del DataFrame para no modificar el original
    df_temp = dataframe.copy()

    # Mostrar información del dataset que se va a procesar
    print(f"Procesando dataset con {len(df_temp):,} registros (2006-2024)")
    print(f"Rango de años en el dataset: {df_temp['Año'].min()} - {df_temp['Año'].max()}")
    print(f"Rango de índices: 0 a {len(df_temp)-1}")

    # Identificar los índices de los valores atípicos
    # Valores menores al mínimo establecido
    indices_menores = df_temp[df_temp[columna] < valor_min].index

    # Valores mayores al máximo establecido
    indices_mayores = df_temp[df_temp[columna] > valor_max].index

    # Combinar todos los índices atípicos
    indices_atipicos = indices_menores.union(indices_mayores)

    print(f"\nSe encontraron {len(indices_atipicos)} valores atípicos:")
    print(f"- {len(indices_menores)} valores menores a {valor_min}°C")
    print(f"- {len(indices_mayores)} valores mayores a {valor_max}°C")

    # Si no hay valores atípicos, retornar el dataframe original
    if len(indices_atipicos) == 0:
        print("¡No se encontraron valores atípicos para reemplazar!")
        return df_temp

    # Mostrar información sobre los años donde se encontraron valores atípicos
    if len(indices_atipicos) > 0:
        años_con_atipicos = df_temp.loc[indices_atipicos, 'Año'].value_counts().sort_index()
        print(f"\nDistribución de valores atípicos por año:")
        for año, conteo in años_con_atipicos.items():
            print(f"  Año {año}: {conteo} valores atípicos")

    # Procesar cada valor atípico
    contador_procesados = 0
    for idx in indices_atipicos:
        print(f"\nProcesando índice {idx}:")
        print(f"  Valor actual = {df_temp.loc[idx, columna]:.2f}°C")
        print(f"  Año del registro = {df_temp.loc[idx, 'Año']}")

        # Verificar que el índice existe en el DataFrame
        if idx not in df_temp.index:
            print(f"  ¡ERROR! Índice {idx} no existe en el DataFrame")
            continue

        # Obtener los 3 índices anteriores y siguientes
        indice_inicio = max(0, idx - 3)  # Evitar índices negativos
        indice_fin = min(len(df_temp) - 1, idx + 3)  # Evitar índices fuera del rango

        # Crear lista de índices para el promedio (excluyendo el valor atípico actual)
        indices_promedio = []

        # Agregar los 3 índices anteriores (si existen)
        for i in range(1, 4):
            indice_anterior = idx - i
            if indice_anterior >= 0 and indice_anterior in df_temp.index:
                indices_promedio.append(indice_anterior)

        # Agregar los 3 índices siguientes (si existen)
        for i in range(1, 4):
            indice_siguiente = idx + i
            if indice_siguiente < len(df_temp) and indice_siguiente in df_temp.index:
                indices_promedio.append(indice_siguiente)

        # Verificar que tenemos al menos un valor para calcular el promedio
        if len(indices_promedio) > 0:
            # Calcular el promedio de los valores seleccionados
            promedio = df_temp.loc[indices_promedio, columna].mean()

            print(f"  Índices usados para el promedio: {indices_promedio}")
            print(f"  Valores vecinos: {df_temp.loc[indices_promedio, columna].tolist()}")
            print(f"  Promedio calculado: {promedio:.2f}°C")

            # Reemplazar el valor atípico con el promedio calculado
            valor_anterior = df_temp.loc[idx, columna]
            df_temp.loc[idx, columna] = promedio

            print(f"  Valor reemplazado: {valor_anterior:.2f}°C -> {promedio:.2f}°C")
            contador_procesados += 1
        else:
            print(f"  ¡ADVERTENCIA! No hay valores vecinos para calcular el promedio en índice {idx}")
            print(f"  El valor atípico {df_temp.loc[idx, columna]:.2f}°C no fue modificado")

    print(f"\n✓ Procesamiento completado: {contador_procesados} valores atípicos reemplazados")

    return df_temp

# =============================================================================
# APLICAR LA FUNCIÓN CORREGIDA AL DATASET FILTRADO (2006-2024)
# =============================================================================

print("=" * 70)
print("INICIANDO PROCESO DE LIMPIEZA DE DATOS ATÍPICOS EN df_filtrado (2006-2024)")
print("=" * 70)

# Mostrar estadísticas antes de la limpieza
print("\nESTADÍSTICAS ANTES DE LA LIMPIEZA:")
print(df_filtrado_reset['ValorObservado'].describe())

# Aplicar la función de reemplazo al dataset filtrado con índices reseteados
df_limpiado = reemplazar_valores_atipicos_filtrado(df_filtrado_reset, 'ValorObservado')

print("\n" + "=" * 70)
print("PROCESO DE LIMPIEZA COMPLETADO")
print("=" * 70)

Se han truncado las últimas 5000 líneas del flujo de salida.
  Promedio calculado: 23.68°C
  Valor reemplazado: 15.10°C -> 23.68°C

Procesando índice 219015:
  Valor actual = 15.10°C
  Año del registro = 2022
  Índices usados para el promedio: [219014, 219013, 219012, 219016, 219017, 219018]
  Valores vecinos: [30.2, 23.683333333333334, 24.7, 28.4, 31.8, 27.7]
  Promedio calculado: 27.75°C
  Valor reemplazado: 15.10°C -> 27.75°C

Procesando índice 219019:
  Valor actual = 14.30°C
  Año del registro = 2022
  Índices usados para el promedio: [219018, 219017, 219016, 219020, 219021, 219022]
  Valores vecinos: [27.7, 31.8, 28.4, 27.8, 24.9, 34.8]
  Promedio calculado: 29.23°C
  Valor reemplazado: 14.30°C -> 29.23°C

Procesando índice 219035:
  Valor actual = 14.30°C
  Año del registro = 2022
  Índices usados para el promedio: [219034, 219033, 219032, 219036, 219037, 219038]
  Valores vecinos: [19.9, 23.3, 29.2, 23.7, 26.7, 24.2]
  Promedio calculado: 24.50°C
  Valor reemplazado: 14.30°C ->

In [ ]:
print(df_limpiado.head())
#para ver cuantas filas hay
print(len(df_limpiado))
# para ver cuales años tengo
print(df_limpiado['Año'].unique())
# para ver cuantas observaciones hay por año
print(df_limpiado['Año'].value_counts())

   CodigoEstacion  CodigoSensor         FechaObservacion          Dia   Año  \
0        28035060            68  2006 abr 01 03:00:00 AM  2006 abr 01  2006   
1        28035070            68  2006 abr 01 02:00:00 PM  2006 abr 01  2006   
2        28035070            68  2006 abr 01 04:00:00 AM  2006 abr 01  2006   
3        28035060            68  2006 abr 01 07:00:00 AM  2006 abr 01  2006   
4        28035070            68  2006 abr 01 01:00:00 PM  2006 abr 01  2006   

     Mes  ValorObservado   NombreEstacion Departamento   Municipio  \
0  abril           25.10  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
1  abril           24.60  GUATAPURI - AUT        CESAR  VALLEDUPAR   
2  abril           26.96  GUATAPURI - AUT        CESAR  VALLEDUPAR   
3  abril           25.80  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
4  abril           26.50  GUATAPURI - AUT        CESAR  VALLEDUPAR   

  ZonaHidrografica    Latitud   Longitud DescripcionSensor UnidadMedida  
0            CESAR  10.463611 

In [ ]:
# ANÁLISIS ADICIONAL Y GUARDADO DEL RESULTADO

# Crear un resumen comparativo
print("\n" + "=" * 60)
print("RESUMEN COMPARATIVO")
print("=" * 60)

print(f"Valores atípicos antes: {len(df[df['ValorObservado'] < 16]) + len(df[df['ValorObservado'] > 42.5])}")
print(f"Valores atípicos después: {len(df_limpiado[df_limpiado['ValorObservado'] < 16]) + len(df_limpiado[df_limpiado['ValorObservado'] > 42.5])}")

print(f"\nTemperatura mínima antes: {df['ValorObservado'].min():.2f}°C")
print(f"Temperatura mínima después: {df_limpiado['ValorObservado'].min():.2f}°C")

print(f"\nTemperatura máxima antes: {df['ValorObservado'].max():.2f}°C")
print(f"Temperatura máxima después: {df_limpiado['ValorObservado'].max():.2f}°C")


RESUMEN COMPARATIVO
Valores atípicos antes: 3852
Valores atípicos después: 0

Temperatura mínima antes: 0.00°C
Temperatura mínima después: 16.00°C

Temperatura máxima antes: 49.60°C
Temperatura máxima después: 42.50°C


In [ ]:
# Mostrar las columnas disponibles en el dataset para verificar los nombres exactos
print("Columnas disponibles en el dataset:")
print(df_limpiado.columns.tolist())
print("\n")

# Ver una muestra de los datos para entender la estructura
print("Primeras 5 filas del dataset:")
print(df_limpiado.head())
print("\n")

Columnas disponibles en el dataset:
['CodigoEstacion', 'CodigoSensor', 'FechaObservacion', 'Dia', 'Año', 'Mes', 'ValorObservado', 'NombreEstacion', 'Departamento', 'Municipio', 'ZonaHidrografica', 'Latitud', 'Longitud', 'DescripcionSensor', 'UnidadMedida']


Primeras 5 filas del dataset:
   CodigoEstacion  CodigoSensor         FechaObservacion          Dia   Año  \
0        28035060            68  2006 abr 01 03:00:00 AM  2006 abr 01  2006   
1        28035070            68  2006 abr 01 02:00:00 PM  2006 abr 01  2006   
2        28035070            68  2006 abr 01 04:00:00 AM  2006 abr 01  2006   
3        28035060            68  2006 abr 01 07:00:00 AM  2006 abr 01  2006   
4        28035070            68  2006 abr 01 01:00:00 PM  2006 abr 01  2006   

     Mes  ValorObservado   NombreEstacion Departamento   Municipio  \
0  abril           25.10  FEDEARROZ - AUT        CESAR  VALLEDUPAR   
1  abril           24.60  GUATAPURI - AUT        CESAR  VALLEDUPAR   
2  abril           26.96  

In [ ]:

print(df_limpiado.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636445 entries, 0 to 636444
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CodigoEstacion     636445 non-null  int64  
 1   CodigoSensor       636445 non-null  int64  
 2   FechaObservacion   636445 non-null  object 
 3   Dia                636445 non-null  object 
 4   Año                636445 non-null  int64  
 5   Mes                636445 non-null  object 
 6   ValorObservado     636445 non-null  float64
 7   NombreEstacion     636445 non-null  object 
 8   Departamento       636445 non-null  object 
 9   Municipio          636445 non-null  object 
 10  ZonaHidrografica   636445 non-null  object 
 11  Latitud            636445 non-null  float64
 12  Longitud           636445 non-null  float64
 13  DescripcionSensor  636445 non-null  object 
 14  UnidadMedida       636445 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 72

In [ ]:
# Guardar primero el DataFrame en un archivo Excel
df_limpiado.to_excel('df_limpiado.xlsx', index=False)

# Luego descargar el archivo
from google.colab import files
files.download('df_limpiado.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =============================================================================
# PASO 1: CREAR LA TABLA RESUMEN POR ESTACIÓN
# =============================================================================

def crear_tabla_estaciones(dataframe):
    """
    Esta función crea una tabla resumen con información de cada estación meteorológica
    incluyendo la frecuencia de observaciones por estación.

    Parámetros:
    - dataframe: DataFrame con los datos de temperatura y estaciones

    Retorna:
    - DataFrame con las columnas: NombreEstacion, CodigoEstacion, Latitud, Longitud, Frecuencia
    """

    print("=" * 70)
    print("INICIANDO CREACIÓN DE TABLA RESUMEN DE ESTACIONES")
    print("=" * 70)

    # Verificar que las columnas necesarias existen en el dataset
    columnas_requeridas = ['NombreEstacion', 'CodigoEstacion', 'Latitud', 'Longitud']
    columnas_faltantes = [col for col in columnas_requeridas if col not in dataframe.columns]

    if columnas_faltantes:
        print(f"¡ADVERTENCIA! Columnas faltantes: {columnas_faltantes}")
        print("No se puede crear la tabla completa.")
        return None

    print(" Todas las columnas requeridas están presentes en el dataset")

    # PASO 2: AGRUPAR POR ESTACIÓN Y CONTAR LA FRECUENCIA
    print("\n1. Agrupando datos por estación y calculando frecuencias...")

    # Crear un grupo por las columnas de identificación de estación
    # Esto nos permite trabajar con cada estación individualmente
    grupo_estaciones = dataframe.groupby(['NombreEstacion', 'CodigoEstacion', 'Latitud', 'Longitud'])

    # Contar el número de observaciones (filas) para cada estación
    # size() cuenta todas las filas en cada grupo
    tabla_resumen = grupo_estaciones.size().reset_index(name='Frecuencia')

    print(f" Se identificaron {len(tabla_resumen)} estaciones diferentes")

    # PASO 3: ORDENAR LA TABLA POR FRECUENCIA (de mayor a menor)
    print("\n2. Ordenando estaciones por frecuencia (descendente)...")
    tabla_resumen = tabla_resumen.sort_values('Frecuencia', ascending=False)

    # PASO 4: CALCULAR ESTADÍSTICAS ADICIONALES SOBRE LAS FRECUENCIAS
    print("\n3. Calculando estadísticas de frecuencias...")
    total_observaciones = tabla_resumen['Frecuencia'].sum()
    frecuencia_promedio = tabla_resumen['Frecuencia'].mean()
    frecuencia_min = tabla_resumen['Frecuencia'].min()
    frecuencia_max = tabla_resumen['Frecuencia'].max()

    print(f"   - Total de observaciones en todas las estaciones: {total_observaciones:,}")
    print(f"   - Frecuencia promedio por estación: {frecuencia_promedio:.1f}")
    print(f"   - Frecuencia mínima: {frecuencia_min}")
    print(f"   - Frecuencia máxima: {frecuencia_max}")

    # PASO 5: MOSTRAR LA DISTRIBUCIÓN DE FRECUENCIAS
    print("\n4. Distribución de frecuencias por estación:")
    print(tabla_resumen['Frecuencia'].describe())

    return tabla_resumen

# APLICAR LA FUNCIÓN PARA CREAR LA TABLA RESUMEN
tabla_estaciones = crear_tabla_estaciones(df_limpiado)

INICIANDO CREACIÓN DE TABLA RESUMEN DE ESTACIONES
 Todas las columnas requeridas están presentes en el dataset

1. Agrupando datos por estación y calculando frecuencias...
 Se identificaron 4 estaciones diferentes

2. Ordenando estaciones por frecuencia (descendente)...

3. Calculando estadísticas de frecuencias...
   - Total de observaciones en todas las estaciones: 636,445
   - Frecuencia promedio por estación: 159111.2
   - Frecuencia mínima: 23667
   - Frecuencia máxima: 439757

4. Distribución de frecuencias por estación:
count         4.000000
mean     159111.250000
std      193476.068405
min       23667.000000
25%       34635.750000
50%       86510.500000
75%      210986.000000
max      439757.000000
Name: Frecuencia, dtype: float64


In [ ]:
# =============================================================================
# PASO 4: MOSTRAR LA TABLA RESULTANTE
# =============================================================================

if tabla_estaciones is not None:
    print("\n" + "=" * 70)
    print("TABLA RESUMEN DE ESTACIONES METEOROLÓGICAS")
    print("=" * 70)

    # Mostrar todas las estaciones encontradas
    pd.set_option('display.max_rows', None)  # Mostrar todas las filas
    pd.set_option('display.width', 1000)     # Ancho de display

    print(tabla_estaciones.to_string(index=False))

    # Resetear opciones de display
    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')


TABLA RESUMEN DE ESTACIONES METEOROLÓGICAS
             NombreEstacion  CodigoEstacion   Latitud   Longitud  Frecuencia
   AEROPUERTO ALFONSO LOPEZ        28025502 10.436167 -73.247667      439757
            FEDEARROZ - AUT        28035060 10.463611 -73.248056      134729
            GUATAPURI - AUT        28035070 10.732139 -73.392417       38292
CARACOLÍ HDA MANATURE - AUT      2802500079 10.088694 -73.731667       23667


In [ ]:
# =============================================================================
# PASO 1: CREAR LA TABLA RESUMEN POR ESTACIÓN (ACTUALIZADA)
# =============================================================================

def crear_tabla_estaciones_actualizada(dataframe):
    """
    Esta función crea una tabla resumen con información de cada estación meteorológica
    incluyendo la frecuencia de observaciones por estación y el código del sensor.

    Parámetros:
    - dataframe: DataFrame con los datos de temperatura y estaciones

    Retorna:
    - DataFrame con las columnas: NombreEstacion, CodigoEstacion, CodigoSensor,
                                  Latitud, Longitud, Frecuencia
    """

    print("=" * 70)
    print("INICIANDO CREACIÓN DE TABLA RESUMEN DE ESTACIONES (ACTUALIZADA)")
    print("=" * 70)

    # Verificar que las columnas necesarias existen en el dataset
    columnas_requeridas = ['NombreEstacion', 'CodigoEstacion', 'CodigoSensor', 'Latitud', 'Longitud']
    columnas_faltantes = [col for col in columnas_requeridas if col not in dataframe.columns]

    if columnas_faltantes:
        print(f"¡ADVERTENCIA! Columnas faltantes: {columnas_faltantes}")
        print("No se puede crear la tabla completa.")
        return None

    print(" Todas las columnas requeridas están presentes en el dataset")

    # PASO 2: AGRUPAR POR ESTACIÓN Y CONTAR LA FRECUENCIA
    print("\n1. Agrupando datos por estación y calculando frecuencias...")

    # Crear un grupo por las columnas de identificación de estación
    # AHORA INCLUYENDO 'CodigoSensor' en el agrupamiento
    grupo_estaciones = dataframe.groupby(['NombreEstacion', 'CodigoEstacion', 'CodigoSensor', 'Latitud', 'Longitud'])

    # Contar el número de observaciones (filas) para cada estación
    tabla_resumen = grupo_estaciones.size().reset_index(name='Frecuencia')

    print(f" Se identificaron {len(tabla_resumen)} estaciones diferentes con sus sensores")

    # PASO 3: ORDENAR LA TABLA POR FRECUENCIA (de mayor a menor)
    print("\n2. Ordenando estaciones por frecuencia (descendente)...")
    tabla_resumen = tabla_resumen.sort_values('Frecuencia', ascending=False)

    # PASO 4: CALCULAR ESTADÍSTICAS ADICIONALES SOBRE LAS FRECUENCIAS
    print("\n3. Calculando estadísticas de frecuencias...")
    total_observaciones = tabla_resumen['Frecuencia'].sum()
    frecuencia_promedio = tabla_resumen['Frecuencia'].mean()
    frecuencia_min = tabla_resumen['Frecuencia'].min()
    frecuencia_max = tabla_resumen['Frecuencia'].max()

    print(f"   - Total de observaciones en todas las estaciones: {total_observaciones:,}")
    print(f"   - Frecuencia promedio por estación: {frecuencia_promedio:.1f}")
    print(f"   - Frecuencia mínima: {frecuencia_min}")
    print(f"   - Frecuencia máxima: {frecuencia_max}")

    # PASO 5: ANÁLISIS DE SENSORES POR ESTACIÓN
    print("\n4. Análisis de sensores por estación:")

    # Verificar si hay estaciones con múltiples sensores
    conteo_sensores_por_estacion = tabla_resumen.groupby('NombreEstacion')['CodigoSensor'].nunique()
    estaciones_con_multiples_sensores = conteo_sensores_por_estacion[conteo_sensores_por_estacion > 1]

    if len(estaciones_con_multiples_sensores) > 0:
        print(f"   - Estaciones con múltiples sensores: {len(estaciones_con_multiples_sensores)}")
        for estacion, num_sensores in estaciones_con_multiples_sensores.items():
            print(f"     * {estacion}: {num_sensores} sensores")
    else:
        print("   - Todas las estaciones tienen un único sensor")

    # Mostrar la variedad de sensores encontrados
    sensores_unicos = tabla_resumen['CodigoSensor'].nunique()
    print(f"   - Total de sensores diferentes identificados: {sensores_unicos}")

    return tabla_resumen

# APLICAR LA FUNCIÓN ACTUALIZADA
tabla_estaciones_actualizada = crear_tabla_estaciones_actualizada(df_limpiado)

INICIANDO CREACIÓN DE TABLA RESUMEN DE ESTACIONES (ACTUALIZADA)
 Todas las columnas requeridas están presentes en el dataset

1. Agrupando datos por estación y calculando frecuencias...
 Se identificaron 5 estaciones diferentes con sus sensores

2. Ordenando estaciones por frecuencia (descendente)...

3. Calculando estadísticas de frecuencias...
   - Total de observaciones en todas las estaciones: 636,445
   - Frecuencia promedio por estación: 127289.0
   - Frecuencia mínima: 23667
   - Frecuencia máxima: 376412

4. Análisis de sensores por estación:
   - Estaciones con múltiples sensores: 1
     * AEROPUERTO ALFONSO LOPEZ: 2 sensores
   - Total de sensores diferentes identificados: 2


In [ ]:
# =============================================================================
# PASO 4: MOSTRAR LA TABLA RESULTANTE ACTUALIZADA
# =============================================================================

if tabla_estaciones_actualizada is not None:
    print("\n" + "=" * 70)
    print("TABLA RESUMEN DE ESTACIONES METEOROLÓGICAS (CON CÓDIGO SENSOR)")
    print("=" * 70)

    # Mostrar todas las estaciones encontradas con la nueva columna
    pd.set_option('display.max_rows', None)  # Mostrar todas las filas
    pd.set_option('display.width', 1200)     # Ancho de display aumentado para la columna extra
    pd.set_option('display.max_columns', None)  # Mostrar todas las columnas

    print(tabla_estaciones_actualizada.to_string(index=False))

    # Resetear opciones de display
    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')
    pd.reset_option('display.max_columns')



TABLA RESUMEN DE ESTACIONES METEOROLÓGICAS (CON CÓDIGO SENSOR)
             NombreEstacion  CodigoEstacion  CodigoSensor   Latitud   Longitud  Frecuencia
   AEROPUERTO ALFONSO LOPEZ        28025502            71 10.436167 -73.247667      376412
            FEDEARROZ - AUT        28035060            68 10.463611 -73.248056      134729
   AEROPUERTO ALFONSO LOPEZ        28025502            68 10.436167 -73.247667       63345
            GUATAPURI - AUT        28035070            68 10.732139 -73.392417       38292
CARACOLÍ HDA MANATURE - AUT      2802500079            68 10.088694 -73.731667       23667


In [ ]:
# =============================================================================
# PROMEDIAR VALORES DIARIOS - DE MÚLTIPLES REGISTROS A UNO SOLO POR DÍA
# =============================================================================

print("=" * 70)
print("PROCESO: PROMEDIAR VALORES DIARIOS")
print("=" * 70)

# Mostrar cómo están actualmente los datos
print("\nESTADO ACTUAL DE LOS DATOS:")
print(f"Total de registros en df_limpiado: {len(df_limpiado):,}")
print(f"Primeras fechas con múltiples registros por día:")

# Verificar cuántos registros hay por día actualmente
conteo_por_dia_actual = df_limpiado['Dia'].value_counts()
print(f"\nEjemplos de días con múltiples registros:")
for dia, conteo in conteo_por_dia_actual.head(10).items():
    print(f"  {dia}: {conteo} registros")

print(f"\nEstadísticas de registros por día:")
print(f"  Máximo registros en un día: {conteo_por_dia_actual.max()}")
print(f"  Mínimo registros en un día: {conteo_por_dia_actual.min()}")
print(f"  Promedio de registros por día: {conteo_por_dia_actual.mean():.2f}")

PROCESO: PROMEDIAR VALORES DIARIOS

ESTADO ACTUAL DE LOS DATOS:
Total de registros en df_limpiado: 636,445
Primeras fechas con múltiples registros por día:

Ejemplos de días con múltiples registros:
  2024 nov 21: 1586 registros
  2024 nov 04: 1586 registros
  2024 nov 15: 1584 registros
  2024 nov 14: 1584 registros
  2024 nov 20: 1584 registros
  2024 nov 11: 1584 registros
  2024 nov 13: 1584 registros
  2024 nov 12: 1584 registros
  2024 nov 22: 1584 registros
  2024 nov 16: 1584 registros

Estadísticas de registros por día:
  Máximo registros en un día: 1586
  Mínimo registros en un día: 1
  Promedio de registros por día: 94.33


In [ ]:
# =============================================================================
# AGRUPAR POR DÍA Y CALCULAR EL PROMEDIO DIARIO (CORREGIDO)
# =============================================================================

print("\n" + "=" * 70)
print("AGRUPAMIENTO Y CÁLCULO DE PROMEDIOS DIARIOS")
print("=" * 70)

def promediar_datos_diarios(dataframe, columna_fecha='Dia', columna_temperatura='ValorObservado'):
    """
    Agrupa los datos por día y calcula el promedio de temperatura para cada día

    Parámetros:
    - dataframe: DataFrame con los datos
    - columna_fecha: Nombre de la columna que contiene las fechas (Dia)
    - columna_temperatura: Nombre de la columna con los valores de temperatura

    Retorna:
    - DataFrame con un solo registro por día y el promedio de temperatura
    """

    print("Agrupando datos por día y calculando promedios...")

    # Crear una copia para no modificar el original
    df_temp = dataframe.copy()

    # AGRUPAR POR FECHA Y CALCULAR EL PROMEDIO
    # Agrupamos por la columna 'Dia' y calculamos la media de temperatura
    df_promediado = df_temp.groupby(columna_fecha, as_index=False).agg({
        columna_temperatura: 'mean',  # Promedio de temperatura
        'Año': 'first',               # Mantener el año (será el mismo para cada fecha)
        'NombreEstacion': lambda x: ', '.join(str(val) for val in x.unique()),  # Convertir a string antes de unir
        'CodigoEstacion': lambda x: ', '.join(str(val) for val in x.unique()),  # Convertir a string antes de unir
        'CodigoSensor': lambda x: ', '.join(str(val) for val in x.unique())     # Convertir a string antes de unir
    })

    # Renombrar la columna de temperatura para indicar que es el promedio
    df_promediado = df_promediado.rename(columns={columna_temperatura: 'TemperaturaPromedio'})

    print("✓ Agrupamiento completado")

    return df_promediado

# Aplicar la función de promediado corregida
df_diario_promediado = promediar_datos_diarios(df_limpiado, 'Dia', 'ValorObservado')

print(f"\nResultado del agrupamiento:")
print(f"Registros originales: {len(df_limpiado):,}")
print(f"Registros después del promediado: {len(df_diario_promediado):,}")
print(f"Reducción: {len(df_limpiado) - len(df_diario_promediado):,} registros eliminados")


AGRUPAMIENTO Y CÁLCULO DE PROMEDIOS DIARIOS
Agrupando datos por día y calculando promedios...
✓ Agrupamiento completado

Resultado del agrupamiento:
Registros originales: 636,445
Registros después del promediado: 6,747
Reducción: 629,698 registros eliminados


In [ ]:
# =============================================================================
# VERIFICACIÓN DE LOS RESULTADOS
# =============================================================================

print("\n" + "=" * 70)
print("VERIFICACIÓN DE LOS DATOS PROMEDIADOS")
print("=" * 70)

# Mostrar las primeras filas del resultado
print("Primeros 10 días con promedios calculados:")
print(df_diario_promediado.head(10))

# Verificar que ahora tenemos un solo registro por día
dias_unicos = df_diario_promediado['Dia'].nunique()
total_registros = len(df_diario_promediado)

print(f"\nVerificación de unicidad:")
print(f"Días únicos en el resultado: {dias_unicos}")
print(f"Total de registros en el resultado: {total_registros}")
print(f"¿Tenemos un solo registro por día? {dias_unicos == total_registros}")

# Estadísticas del promedio diario
print(f"\nEstadísticas de la temperatura promedio diaria:")
print(df_diario_promediado['TemperaturaPromedio'].describe())


VERIFICACIÓN DE LOS DATOS PROMEDIADOS
Primeros 10 días con promedios calculados:
           Dia  TemperaturaPromedio   Año                    NombreEstacion  \
0  2006 abr 01            25.604375  2006  FEDEARROZ - AUT, GUATAPURI - AUT   
1  2006 abr 02            24.620833  2006  GUATAPURI - AUT, FEDEARROZ - AUT   
2  2006 abr 03            24.989362  2006  GUATAPURI - AUT, FEDEARROZ - AUT   
3  2006 abr 04            25.453191  2006  FEDEARROZ - AUT, GUATAPURI - AUT   
4  2006 abr 05            23.345833  2006  GUATAPURI - AUT, FEDEARROZ - AUT   
5  2006 abr 06            24.787500  2006  FEDEARROZ - AUT, GUATAPURI - AUT   
6  2006 abr 07            25.150000  2006  FEDEARROZ - AUT, GUATAPURI - AUT   
7  2006 abr 08            25.766667  2006  FEDEARROZ - AUT, GUATAPURI - AUT   
8  2006 abr 09            23.514583  2006  GUATAPURI - AUT, FEDEARROZ - AUT   
9  2006 abr 10            24.227083  2006  FEDEARROZ - AUT, GUATAPURI - AUT   

       CodigoEstacion CodigoSensor  
0  28035060

In [ ]:
# para ver "ValorObservado" por día
print(df_diario_promediado[['Dia', 'TemperaturaPromedio']])

              Dia  TemperaturaPromedio
0     2006 abr 01            25.604375
1     2006 abr 02            24.620833
2     2006 abr 03            24.989362
3     2006 abr 04            25.453191
4     2006 abr 05            23.345833
...           ...                  ...
6742  2024 sep 26            26.658737
6743  2024 sep 27            27.805342
6744  2024 sep 28            27.432258
6745  2024 sep 29            26.122162
6746  2024 sep 30            26.909812

[6747 rows x 2 columns]


In [ ]:
# ver base "df_diario_promediado"
print(df_diario_promediado)

# para descriptivo de la variable "TemperaturaPromedio"
print(df_diario_promediado['TemperaturaPromedio'].describe())

# para descargar la base "df_diario_promediado" en formato excel
from google.colab import files
files.download("df_diario_promediado.xlsx")
df_diario_promediado.to_excel('df_diario_promediado.xlsx', index=False)


              Dia  TemperaturaPromedio   Año  \
0     2006 abr 01            25.604375  2006   
1     2006 abr 02            24.620833  2006   
2     2006 abr 03            24.989362  2006   
3     2006 abr 04            25.453191  2006   
4     2006 abr 05            23.345833  2006   
...           ...                  ...   ...   
6742  2024 sep 26            26.658737  2024   
6743  2024 sep 27            27.805342  2024   
6744  2024 sep 28            27.432258  2024   
6745  2024 sep 29            26.122162  2024   
6746  2024 sep 30            26.909812  2024   

                        NombreEstacion      CodigoEstacion CodigoSensor  
0     FEDEARROZ - AUT, GUATAPURI - AUT  28035060, 28035070           68  
1     GUATAPURI - AUT, FEDEARROZ - AUT  28035070, 28035060           68  
2     GUATAPURI - AUT, FEDEARROZ - AUT  28035070, 28035060           68  
3     FEDEARROZ - AUT, GUATAPURI - AUT  28035060, 28035070           68  
4     GUATAPURI - AUT, FEDEARROZ - AUT  28035070, 280

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>